**Test 2BL: Ley de Benford**

El test de Benford para el segundo dígito es un método estadístico que resulta útil en la detección de anomalías o manipulación de resultados electorales [[mebane2006]](https://www.researchgate.net/publication/237341643_Election_Forensics_Vote_Counts_and_Benford's_Law).

En el contexto de anális forense de elecciones, el test 2BL (*2nd digit Benford's Law Test* o en español test del segundo dígito de la ley de Benford) consiste en comparar la distribución del segundo dígito más significativo del conteo de votos en recintos con la distribución generalizada de Benford para el segundo dígito.

La distribución generalizada de Benford se adapta a la distribución natural de datos de elecciones.

La distribución $2BL$ usa las siguientes frecuencias $q_0,\ldots,q_9$ de los dígitos $0,\ldots,9$:

$$
(q_0,\ldots,q_9) = (0.120, 0.114, 0.109, 0.104, 0.100, 0.097, 0.093, 0.090, 0.088, 0.085)
$$

En los siguientes diagramas comparamos el la distribución $2BL$ con la distribución calculada de los conteos por partido.

Sólo consideramos los partidos "CC", "MAS - IPSP", "21F" Y "PDC", porque los demás partidos no alcanzan valores lo suficientemente altos en un número suficiente de actas para ser considerado en el test.

Se evidencia que los conteos de votos correspondientes a los partidos "CC" y "MAS - IPSP", tanto en elecciones presidenciales como para diputados, presentan una distribución aleatoria uniforme, mientras que los partidos "21F" y "PDC" presentan una tendencia decreciente.

La distribución aleatoria uniforme evidenciada en los conteos de votos de "CC" y ""MAS - IPSP" representa un fallo claro del test $2BL$. 

In [157]:
import pandas as pd
import os
from os.path import join, exists
import plotly.graph_objects as go
import ipywidgets as widgets

data_path = join(os.getcwd(),"data")
csv_filename = "acta.2019.10.25.21.09.30.csv" 

df = pd.read_csv(join(data_path,csv_filename))

partidos = ['CC','MAS - IPSP','21F','PDC']
elecciones = ["Presidente y Vicepresidente", "Diputados Uninominales"]
q_is = [0.12, 0.114, 0.109, 0.104, 0.1, 0.097, 0.093, 0.09, 0.088, 0.085]

def count_second_digit(eleccion, partido):
    edf = df[(df[partido] >= 10)&(df["Elección"] == elecciones[0])].copy()

    second_digit_counts = {digit: 0 for digit in range(10)}

    for val in edf[partido]:
        second_digit_counts[val % 10] += 1
        
    return (edf.shape[0], list(second_digit_counts.values()))

def get_chi_square_2bl(n, n_is):
    return(sum([(n_i - (n * q_i))**2 / (n * q_i) for n_i,q_i in zip(n_is,q_is)]))

def get_chi_square_uniform(n, n_is):
    return(sum([(n_i - (n / 10))**2 / (n / 10) for n_i in n_is]))

n, n_is = count_second_digit(elecciones[0],partidos[0])
chi_2bl = get_chi_square_2bl(n, n_is)
chi_uniform = get_chi_square_uniform(n, n_is)

partidos_dropdown = widgets.Dropdown(value="CC",options=partidos,description="Partido:")
eleccion_dropdown = widgets.Dropdown(value="Presidente y Vicepresidente",options=elecciones,description="Elección:")
figure_widget = go.FigureWidget()
figure_widget.layout.title = "Distribución 2do Dígito"
figure_widget.layout.xaxis = dict(title="Valor 2do dígito", dtick=1)
figure_widget.layout.yaxis = dict(title="Frecuencia")
figure_widget.add_scatter(y=q_is, name="Benford")
figure_widget.add_bar(y=[n_i / n for n_i in n_is], name=partidos[0])
n_html = widgets.HTML(value=f"Conteos considerados: {n}")
chi_2bl_label = widgets.Label(value=r"$\chi^{2}_{2BL}$" + " = " + "{:.2f}".format(chi_2bl))
chi_uniform_label = widgets.Label(value=r"$\chi^{2}_{U}$" + " = " + "{:.2f}".format(chi_uniform))

def update_figure(eleccion,partido):
    n, n_is = count_second_digit(eleccion,partido)
    chi_2bl = get_chi_square_2bl(n, n_is)
    chi_uniform = get_chi_square_uniform(n, n_is)
    figure_widget.data[1].y = [n_i / n for n_i in n_is]
    figure_widget.data[1].name = partido
    n_html.value = f"Conteos considerados: {n}"
    chi_2bl_label.value = r"$\chi^{2}_{2BL}$" + " = " + "{:.2f}".format(chi_2bl)
    chi_uniform_label.value = r"$\chi^{2}_{U}$" + " = " + "{:.2f}".format(chi_uniform)

def on_eleccion_dropdown_change(change):
    eleccion = change["new"]
    partido = partidos_dropdown.value
    update_figure(eleccion,partido)

def on_partidos_dropdown_change(change):
    partido = change["new"]
    eleccion = eleccion_dropdown.value
    update_figure(eleccion,partido)

eleccion_dropdown.observe(on_eleccion_dropdown_change,names="value")
partidos_dropdown.observe(on_partidos_dropdown_change,names="value")

widgets.VBox([
    widgets.HBox([eleccion_dropdown,n_html,]),
    widgets.HBox([partidos_dropdown,chi_2bl_label, chi_uniform_label]),
    figure_widget
])

**Comparación de distribuciones chi-cuadrática ($\chi^2$)**

Para comparar dos distribuciones es comúm usar expresiones del tipo *chi-cuadráticos* ($\chi^2$).

Para la distribución $2BL$ especificada por los valores $q_i$, considerando $n$ conteos de votos, Si $n_i$ representa el número de ocurrencias del segundo dígito $i$ en las actas consideradas, la expresión $\chi^2_{2BL}$ es la siguiente:

$$
\chi^2_{2BL} = \sum_{i=0}^{9} \frac{(n_i-nq_i)^2}{nq_i}
$$

Equivalentemente, consideramos la distribución aleatoria uniforme, cuya expresión chi-cuadrática $\chi^2_{U}$ es la siguiente:

$$
\chi^2_{U} = \sum_{i=0}^{9} \frac{(n_i-n/10)^2}{n/10}
$$

Con un nivel de significancia de $\alpha = 0.05$, el valor crítico de la expresión es de 16.9.  

De esta manera que podemos evidenciar que la distribución de segundos dígitos del partido "MAS - IPSP" y la distribución aleatoria uniforme están significativamente relacionadas, tanto en elecciones presidenciales como en elecciones para diputados.

Los segundos dígitos de ningún partido se encuentra significativamente relacionado con la distribución Benford. 

También se observa que los segundos dígitos de los partidos "CC" y "MAS - IPSP" están más relacionados con la distribución aleatoria que con la distribución Benford, mientras que los segundos dígitos de los partidos "21F" y "PDC" se encuentran más relacionados con la distribución Benford que con la distribución aleatoria uniforme.  